#Importando dados

Começamos importando todos os dados necessarios para nosso treinamento, assim como as bibliotecas que serão usadas:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


x_data = 